<a href="https://colab.research.google.com/github/KoreTane/PredictiveLeadManagement/blob/main/Forecasting_marketing_investments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings

In [2]:
#dfm = pd.read_excel('MatriculaCEP.xlsx')
#dfc = pd.read_excel('fluxo_caixa_cbs.xlsx')
df = pd.read_excel('CrmAgendaClientes.xlsx')

In [3]:
# Extrair a coluna a partir do 2 delimitador '#'
#dfc['Matricula'] = dfc['Descrição'].str.split('#', expand=True)[2]
# Remover os caracteres '#\d+' da coluna 'Descrição'
#dfc['Aluno'] = dfc['Descrição'].str.replace(r'#\d+', '', regex=True)
#dfc = dfc[['Matricula', 'Data', 'Aluno', 'Valor', 'Tipo']]
warnings.filterwarnings('ignore')

In [4]:
# Dados dos investimentos em MKT
ano = ['2023','2023','2023','2023','2024','2024']
trimestre = ['1','2','3','4','1','2']
cliques = [303, 1179, 1654, 841, 1666, 812]
impressoes = [9917, 29871, 51773, 27408, 37411, 12657]
cpc_medio = ['0,60', '0,66', '0,55', '0,73', '0,55', '0,44']
custo = ['181,82', '780,94', '917,74', '612,22', '919,50', '357,89']

# Criar DataFrame
dfm = pd.DataFrame({
    'Ano':ano,
    'Trimestre': trimestre,
    'Cliques': cliques,
    'Impressões': impressoes,
    'CPC Médio': cpc_medio,
    'Custo': custo
})

In [5]:
df['SITUAÇÃO_FINAL'].value_counts()

SITUAÇÃO_FINAL
DESISTIU       1247
MATRICULADO     277
EM ABERTO       142
Name: count, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1666 entries, 0 to 1665
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA 1º CONTATO   1666 non-null   datetime64[ns]
 1   TRIMESTRE         1666 non-null   int64         
 2   NOME COMPLETO     1664 non-null   object        
 3   IDADE             1666 non-null   int64         
 4   FAIXA ETARIA      1666 non-null   object        
 5   GENERO            1666 non-null   object        
 6   TELEFONE          1666 non-null   object        
 7   STATUS            1666 non-null   object        
 8   VIA               1666 non-null   object        
 9   ESTILOS           1666 non-null   object        
 10  DATA AGENDAMENTO  1666 non-null   object        
 11  TURNO             1666 non-null   object        
 12  PROFESSOR         1666 non-null   object        
 13  COMPARECEU        1666 non-null   object        
 14  SITUAÇÃO_FINAL    1666 n

In [7]:
#df[df['DATA'].dt.year == 2023]

In [13]:
# Filtrar o DataFrame para incluir apenas as linhas que correspondem ao trimestre 1 do ano 2023 e ao tipo de contato "1º Contato"
df_filtered = df[(df['TRIMESTRE'].isin([1, 2, 3, 4])) & (df['DATA 1º CONTATO'].dt.year == 2023)]# & (df['SITUAÇÃO_FINAL']== 'MATRICULADO')]

# Contar a quantidade de situações finais em cada tipo
situacao_final_counts = df_filtered['SITUAÇÃO_FINAL'].value_counts()
situacao_final_counts

SITUAÇÃO_FINAL
DESISTIU       411
EM ABERTO      102
MATRICULADO     86
Name: count, dtype: int64

In [12]:
df[(df['TRIMESTRE'].isin([1, 2, 3, 4])) & (df['DATA 1º CONTATO'].dt.year == 2023)]

,DATA 1º CONTATO,TRIMESTRE,NOME COMPLETO,IDADE,FAIXA ETARIA,GENERO,TELEFONE,STATUS,VIA,ESTILOS,DATA AGENDAMENTO,TURNO,PROFESSOR,COMPARECEU,SITUAÇÃO_FINAL
979,2023-01-02,1,ROSEMARY DIAS,19,ADULTO,MASCULINO,0000-0000,CAPTAÇÃO,WHATSAPP,JAZZ FUNK,Não agendou,NOITE,ALISSON,NÃO,DESISTIU
980,2023-01-02,1,CAROLINE CAETANO,19,ADULTO,MASCULINO,0000-0000,CAPTAÇÃO,WHATSAPP,JAZZ FUNK,Não agendou,NOITE,ALISSON,NÃO,EM ABERTO
981,2023-01-03,1,LUMA,19,ADULTO,FEMININO,0000-0000,CAPTAÇÃO,WHATSAPP,JAZZ FUNK,Não agendou,NOITE,ALISSON,NÃO,DESISTIU
982,2023-01-03,1,SIMONE MOREIRA P FILHA,19,ADULTO,FEMININO,0000-0000,CAPTAÇÃO,WHATSAPP,JAZZ FUNK,Não agendou,NOITE,ALISSON,NÃO,DESISTIU
983,2023-01-03,1,SABRINA,19,ADULTO,FEMININO,0000-0000,CAPTAÇÃO,WHATSAPP,JAZZ FUNK,Não agendou,NOITE,NÃO INFORMADO,NÃO,DESISTIU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,2023-12-22,4,BEATRIZ SCHIMIDT BARONI JÁ É ALUNA,14,ADOLESCENTE,MASCULINO,0000-0000,AULA EXPERIMENTAL,WHATSAPP,K-POP,45299,NOITE,ALISSON,SIM,MATRICULADO
1575,2023-12-22,4,FELICIA SCHIMIDT,53,ADULTO,FEMININO,0000-0000,AULA EXPERIMENTAL,WHATSAPP,JAZZ,45299,NOITE,LARISSA,SIM,DESISTIU
1611,2023-01-17,1,MARIANA,26,ADULTO,FEMININO,0000-0000,AULA EXPERIMENTAL,WHATSAPP,K-POP,45329,NOITE,ALISSON,NÃO,DESISTIU
1612,2023-01-17,1,ANA,30,ADULTO,FEMININO,0000-0000,AULA EXPERIMENTAL,WHATSAPP,K-POP,45329,NOITE,ALISSON,NÃO,DESISTIU


In [9]:
dfm

,Ano,Trimestre,Cliques,Impressões,CPC Médio,Custo
0,2023,1,303,9917,"0,60","181,82"
1,2023,2,1179,29871,"0,66","780,94"
2,2023,3,1654,51773,"0,55","917,74"
3,2023,4,841,27408,"0,73","612,22"
4,2024,1,1666,37411,"0,55","919,50"
5,2024,2,812,12657,"0,44","357,89"


In [ ]:
# Selecionar as colunas relevantes
X = df[['Custo']]
y = df['SITUAÇÃO_FINAL']

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar um modelo de regressão linear
model = LinearRegression()

# Treinar o modelo com os dados de treinamento
model.fit(X_train, y_train)

# Fazer previsões com os dados de teste
y_pred = model.predict(X_test)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse:.2f}')

# Criar um modelo para prever a conversão com base no valor investido em Custo
def predict_conversion(custo):
    return model.predict([[custo]])

# Testar o modelo com um valor de Custo
custo = 500
conversion = predict_conversion(custo)
print(f'Previsão de conversão com Custo {custo}: {conversion:.2f}')